# Rubrics
Rubrics are one of the two core models of the Turing SDK. Understanding rubrics is critical to being able to leverage the package to get the most value out of the LLM. This notebook focuses entirely on Rubrics. It seeks to
1. **Define the usage and purpose of Rubrics in the SDK**
2. **Explain different methods for defining rubrics**

## Usage and Purpose of Rubrics
The `Rubric` object is used internally by Turing to define how a question should be graded. In our backend, we use the rubric to configure our LLM requests, in order to give you the most accurate and relevant grading and feedback generations. Therefore, the quality of your feedback depends on how well your rubric is suited to the needs of the question. If the LLM is instructred to use a rubric that isn't a great fit for the question, it may give unexpected and undesireable results. As a consequence, it follows that `Rubric` management is a very important aspect of maximizing Turing. 

### Design
Before we cover Rubrics, it will be valuable to gain an understanding of the lower level components of the Rubric ecosystem.

**Objective**

This is the most primitive object in the `Rubric` dependency tree. The `Objective` enum class defines different grading objetives for a question. This class is used by the lower level LLM layers of the Turing architecture to configure the prompt sent to the LLM. The class seeks to define a semi-exhaustive list of objective criteria that one might wish to grade a question by (custom objective definitions coming soon). 

**GradingCriteria**

On top of knowing what to grade for, the LLM also needs to know how much weight to assign each objective. This is where the `GradingCriteria` object steps in. This class provides a clean wrapper that can pair `Objective`s with their corresponding point weight values. The class inherits from pydantic's `BaseModel`, to validate the value of the weight attribtue. The weight must be an increment of 0.5. Increments smaller than this can confuse the LLM when assigning partial credit. Additionally, the pydantic validation ensures that our `GradingCrtiera` is instantiated with an `Objective` object, which ensures that the LLM will know how to grade the objective.

**RubricType**

Finally, we know that a lot of our clients may not need to define granular rubrics for each question, and they might have simply want to apply more general criteria to their questions. This is typically the case when scoring the question is less important than getting subjective feedback for the student. The `RubricType` class is a custom enum class that predefines sets of GradingCriteria for different use cases. 

For example, the `RubricType.FACTUAL_RUBRIC`, can be used to define a rubric that grades for the student's factual understanding, use of evidence, and clarity of writing, each with a point-weight value of 1 point.

**Rubric**

Now that we have defined the lower level components, we can look at the `Rubric` object. They are made up of a few core properties:
- `criteria` (Dict[`Objective`, `GradingCriteria`]): The various grading criteria for the rubric
- `rubric_type` (`RubricType`): The type of the rubric
- `size` (int): The number of GradingCriteria defined for the rubric

The `criteria` dictonary stores the objectives and their corresponding `GradingCriteria`. By using the enum value as our keys to the dictonary, we can access different grading criteria of the rubric by their `objective` properties. This allows us to tinker with the weights of each `criteria` member indivdually. The `rubric_type` property will simply return the `RubricType` used for creating the `Rubric`. The `size` property will return the number of criteria that the rubric will evaluate for (size of `criteria` dictionary). 

The `Rubric` class has a simple API that seeks to provide clients with flexibility in their implementations. We will cover the methods used for object creation in the next section. The most noteable method is the `add_criteria` method. Through this method, we can add additional criteria to a standard `RubricType` generated rubric, or we can define an empty rubric and then add each of our criteria incrementally. 

## Different Methods for Defining Rubrics
The notion of `Rubrics` posed the largest implementation challenge for the Turing SDK. We had to come up with elegant solutions for integrating Turing rubrics with the varied use cases and architechtures of our clients. To do this, we have provided three helpful APIs to the `Rubric` class which can be used for creating clean `Rubric`s.
1. `from_rubric_type` class method
    - This method involves declaring a rubic type from the provided `RubricType` enum class. Each type has its own predefined set of `GradingCriteria` which are used by the LLM to determine how to grade questions
3. From `Objective` objects and assigning them weights
    - This method defining individual `Objectives` and their corresponding point weights, then incrementally adding them to the rubric
2. From data payloads
    - This method inolves deserializing a python object and building the `GradingCriteria` objects directly from the payload

Each of these methods has its own advantages and its own use cases. Let's walk through each of them in more detail.

### Defining rubrics using the `from_rubric_type` class method
This is the most straightforward approach. It leverages the packages predefined rubric templates to quickly and easily instruct the LLM on how a question should be graded, without the need for extensive coupling between Turing's SDK and your pre-existing architechture. This method relies on the `RubricType` enum class. This class defines a set of predefined rubric types, along with their corresponding `Objectives` and point weights. 

As you can see, this method allows you to completely ignore the lower-level `Objetive` and point-weight objects. This approach works well if you are looking for a quick, simple, and seamless integration of the package, or if you are just looking to test out the LLM with a few sample questions.

#### Example

In [ ]:
from turing import Rubric, RubricType

rubric_type = RubricType.FACTUAL_RUBRIC
rubric = Rubric.from_rubric_type(rubric_type)

print(rubric.rubric_type) # RubricType.FACTUAL_RUBRIC
print(rubric.size) # 3 (the three grading criteria defined by the factual rubric)

### Defining rubrics using `Objective` objects and custom point weights
This approach takes one step towards added complexity. We can operate on the lower level `Objective` objects, to create a more granular definition of how our question should be graded by the LLM. In contrast with the previous method, instead of selecting from a curated list of popular `Objective` and weight values, we are now defining our `Objective` and weight values individually, to gain more control over how the LLM process our grading request.

We can call the `add_criteria` method on the the `Rubric` class to add `Objetive`/point-weight paris individually. Under the hood, this creates `GradingCriteria` objects, and adds those objects to the `Rubric` class under the `criteria` attribute.

#### Example

In [ ]:
from turing import Rubric, Objective

rubric = Rubric.empty() # Creates a rubric with no grading criteria

rubric.add_criteria(Objective.FACTUAL, 1.0) # Adds a "factual understanding" objective with a point weight value of 1.0
rubric.add_criteria(Objective.EVIDENCE, 1.0) # Adds a "use of evidence" objective with a point weight value of 1.0
rubric.add_criteria(Objective.ANALYSIS, 1.5) # Adds a "analytical skills" objective with a point weight value of 1.5
rubric.add_criteria(Objective.CLARITY, 0.5) # Adds a "clarity of writing" objective with a point weight value of 0.5

print(rubric.rubric_type) # RubricType.CUSTOM
print(rubric.size) # 4 (the four grading criteria defined by the factual rubric)

### Defining rubrics using the `from_dict` class method
This approach is the most customizable of the three. In this implementation, we can define a custom parser for parsing python dictonaries into `Rubric` objects. This implementation provides the highest level of tuneability based on your internal datamodels. It allows clients to bake the Turing SDK into their design architechture, without the need for extensive schema coupling with the Turing models. 

We can use the `from_dict` method out of the box, to implement a rubric factory that accepts serialized `Objective`/point-weight pairs to create our rubric

### Example

In [ ]:
data = {
    "factual understanding": 1.0, # Adds a Objective.FACTUAL criteria with a point weight value of 1.0
    "use of evidence": 1.0, # Adds a Objective.EVIDENCE cirteria with a point weight value of 1.0
    "analytical skills": 1.5, # Adds a Objective.ANALYSIS criteria with a point weight value of 1.5
    "clarity of writing": 0.5 # Adds a Objective.CLARITY criteria with a point weight value of 0.5
}

rubric = Rubric.from_dict(data) # Creates a rubric from a dictionary

print(rubric.rubric_type) # RubricType.CUSTOM
print(rubric.size) # 4 (the four grading criteria defined by the factual rubric)

# NOTE: We can see that creating this implementation creates the same exact rubric as the prior example.

While this is great, many use cases may want to be able to build rubrics from custom data payloads. This can be achieved by creating a custom `Rubric` class, and overriding the `from_dict` method, to accept payloads of your desired schema. For example, let's say that you have question tags, but the tags you use in your database schema have different values than the `Objective` class of Turing's SDK. Let's look at how we would implement the `Rubric` object in a use case like this

#### Example

In [ ]:
from typing import Dict, Union, List

from turing import Rubric

data = {
    'topic': 'history', # Topic of the question
    'tags': ['analytical', 'evidence', 'accuracy'] # Tags for categorizing a question
}

class CustomRubric(Rubric):

    @staticmethod
    def map_tag_to_objective(tag:str) -> Objective:
        """Resolve a question tag to a grading objective."""
        match tag:
            case 'analytical':
                return Objective.ANALYSIS
            case 'evidence':
                return Objective.EVIDENCE
            case 'accuracy':
                return Objective.FACTUAL
                # ...more cases...
            case _:
                raise ValueError("Invalid Tag!")

    def from_dict(cls, payload: Dict[str, Union[str, List[str]]])
        """Override the default `from_dict` method, to provide custom parsing logic."""
        # Transform our tags into objectives
        objectives = [
            cls.map_tag_to_objective(tag)
            for tag in payload['tags']
        ]

        # Create an empty rubric for us to append values to
        rubric = cls.empty()

        # Add each objective with a point weight value of 1.0
        for obj in objective:
            rubric.add_criteria(obj, 1.0)
        return rubric

rubric = CustomRubric.from_dict(data) # Creates a rubric from a dictionary
print(rubric.rubric_type) # RubricType.CUSTOM
print(rubric.size) # 3 (the three grading criteria defined by the custom rubric)


# Of course, in some cases, we may want an even simpler implementation, that leverages the `from_rubric_type` method.
# Let's take a look at another implementation of the `Rubric` class that uses the `RubricType` API to create a rubric.
class SimpleCustomRubric(Rubric):

    @staticmethod
    def map_topic_to_rubric(self, topic: str) -> RubricType:
        """Resolve a topic string to a rubric type."""
        match topic:
            case 'history':
                return RubricType.ANALYTICAL_RUBRIC
            case 'business':
                return RubricType.APPLICATION_RUBRIC
            case 'math':
                return RubricType.FACTUAL_RUBRIC
            case _:
                raise ValueError("Invalid Topic!")

    @classmethod
    def from_dict(cls, data: Dict[str, Union[str, List[str]]]):
        """Override the default `from_dict` method, to provide custom parsing logic."""
        rubric_type = cls.map_topic_to_rubric(data['topic'])
        return cls.from_rubric_type(rubric_type)

print(rubric.rubric_type) # RubricType.ANALYTICAL_RUBRIC
print(rubric.size) # 3 (the three grading criteria defined by the analytical rubric)

As this example demonstrates, this method of defining a custom `from_dict` class method can be mixed and matched with the above methods in order to fully customize the Rubric creation process. The complexity of the Turing SDK in your application is defined by the amount of coupling you want it to have with your internal structure and data schemas.

## Conclusion

As this notebook shows, rubrics have a very flexible and customizable interface, to allow them to be implemented in all sorts of use cases.